In [95]:
import API_flightapi as API
import pandas as pd
import json
from pprint import pprint


io = open("requetes/flightapi_schedule.json","r")
data = json.load(io)

airport = pd.DataFrame(data['legs'])
price = pd.DataFrame(data['fares'])

def billet_id_split(df):
    temp = df.str.split(pat=':', n=- 1, expand=True)
    result = temp[1] + '-' + temp[2]
    result = result.str.split(pat='-0', n=-1, expand=True)[0]
    
    return result
    

airport['join_id'] = billet_id_split(airport['id'])
price['join_id'] = price['tripId'].str.split(pat=':', n=- 1, expand=True)[1]

Jointure = airport.merge(right = price, on='join_id', how = 'inner')

price = Jointure['price']

price = price.tolist()
        
for i, index in enumerate(price):
    price[i] = price[i]['totalAmount']

Jointure['price'] = price

(Jointure.head(5)).to_json(r'requetes/test_jointure.json')
